In [ ]:
import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

import datetime
import pytz

from astral import LocationInfo
from astral.sun import sun

import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)

In [ ]:
sleep_df = pd.read_parquet('in_bed_metrics_using_cluster.parquet', engine='pyarrow')
ids = sleep_df['participant_id'].unique()
len(ids)

In [ ]:
print(len(sleep_df))

In [ ]:
light = pd.read_parquet('all_participants_light_clean_data_20_JAN_2024.parquet', engine='pyarrow')
light.head(3)

In [ ]:
def prepare_for_stats( df, location, participant):
    df = df[(df['location_name']==location)&(df['participant_id']==participant)].reset_index(drop=True)
    df['date'] =  df['datetime'].dt.date

    df.index = pd.DatetimeIndex(df['datetime'])
    df = df.drop(columns=['datetime'])
    df = df.resample('1T').ffill()
    df = df.rename_axis('datetime').reset_index()
    df = df[df['remove_value?']=='no']
#     df['logvalue'] = np.log10(df['value']+1)
    return df.reset_index(drop=True)



def append_row(df, row):
    return pd.concat([ df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)
    
def season_metereological(date):
    year = str(date.year)
    seasons = {'around-spring-equinox': pd.date_range(start=year+'-02-04', end=year+'-05-05', tz='UTC'),
               'around-summer-solstice': pd.date_range(start=year+'-05-06', end=year+'-08-06', tz='UTC'),
               'around-autumn-equinox': pd.date_range(start=year+'-08-07', end=year+'-11-07', tz='UTC')}
    if date.round('1D') in seasons['around-spring-equinox']:
        return 'around-spring-equinox'
    if date.round('1D') in seasons['around-summer-solstice']:
        return 'around-summer-solstice'
    if date.round('1D') in seasons['around-autumn-equinox']:
        return 'around-autumn-equinox'
    else:
        return 'around-winter-solstice'

In [ ]:
def start_row(participant_id, room, daily_data, date,cluster_number):
    if len(daily_data)/60 == 24:
        
            start_row = {'participant_id': participant_id,'room': room,'tib_cluster_number':cluster_number,
                                            'date':date, 
                                             'daily_start':daily_data['datetime'].iloc[0], 
                                             'metereological_season':'season', 
                                            'mean_daily_light': daily_data['value'].mean(), 
                                            'geometric_mean_daily_light': np.power(10,daily_data['logvalue'].mean()),
                                        'hours_below_10':len(daily_data[daily_data['value']<10])/60,
                                        'hours_10-100':len(daily_data[(daily_data['value']>=10)&(daily_data['value']<100)])/60,
                                        'hours_100-500':len(daily_data[(daily_data['value']>=100)&(daily_data['value']<=500)])/60,
                                        'hours_over_500':len(daily_data[daily_data['value']>500])/60,
                                        'hours_over_1000':len(daily_data[daily_data['value']>1000])/60,
                                        'amplitude':(daily_data['value'].max()-daily_data['value'].min())}

    else:
        start_row = {'participant_id': participant_id,'room': room,'tib_cluster_number':cluster_number,
                            'date':date, 
                            'daily_start':np.nan, 
                             'metereological_season':'season', 
                            'mean_daily_light': np.nan, 
                            'geometric_mean_daily_light': np.nan,
                            'hours_below_10':np.nan,
                            'hours_10-100':np.nan,
                            'hours_100-500':np.nan,
                            'hours_over_500':np.nan,
                            'hours_over_1000':np.nan,
                             'amplitude':np.nan}
                       
    return start_row

def half_light_values(data):
    if len(data)/60 == 24:
        data['cumulative_value'] = data['value'].cumsum()
        data['cumulative_logvalue'] = data['logvalue'].cumsum()

        idx = data['cumulative_value'].sub(data['cumulative_value'].max()/2).abs().idxmin()
        log_idx = data['cumulative_logvalue'].sub(data['cumulative_logvalue'].max()/2).abs().idxmin()

        start_row = { 'half_light': data['cumulative_value'].iloc[idx],
                                            'time_half_light': data['datetime'].iloc[idx],
                                            'half_loglight': data['cumulative_logvalue'].iloc[log_idx],
                                            'time_half_loglight': data['datetime'].iloc[log_idx]}
    else:
        start_row = {'half_light': np.nan,
                            'time_half_light':np.nan,
                            'half_loglight': np.nan,
                            'time_half_loglight':np.nan}
                       
    return start_row


def compute_ntib_averages(ntib_data,time_in_bed_period):
    if len(ntib_data)/60 == time_in_bed_period:
        return {'tib_mean_light': ntib_data['value'].mean(), 
               'tib_geometric_mean_light':np.power(10,ntib_data['logvalue'].mean())}
    else:
        return {'tib_mean_light': np.nan, 
               'tib_geometric_mean_light':np.nan}


def compute_before_ntib_averages(data):
    if len(data)==180:
        return {'before_tib_mean_light': data['value'].mean(), 
               'before_tib_geometric_mean_light':np.power(10,data['logvalue'].mean())}
    else:
        return {'before_tib_mean_light': np.nan, 
               'before_tib_geometric_mean_light':np.nan}

def compute_after_ntib_averages(data):
    if len(data)==180:
        return {'after_tib_mean_light': data['value'].mean(), 
           'after_tib_geometric_mean_light':np.power(10,data['logvalue'].mean())}
    else:
        return {'after_tib_mean_light': np.nan, 
           'after_tib_geometric_mean_light':np.nan}


def compute_light_stats(daily_averages, halfl_averages, ntib_averages,bf_ntib_averages, af_ntib_averages):
    merged_dict = {**daily_averages,**halfl_averages,**ntib_averages,**bf_ntib_averages,**af_ntib_averages}
    new_row = pd.Series(merged_dict)
    return new_row

In [ ]:
def start_light_table():
    return pd.DataFrame(columns=['participant_id', 'room', 'date', 'tib_cluster_number',
                                 'daily_start','metereological_season', 
                                 'mean_daily_light', 'geometric_mean_daily_light', 'half_light', 'time_half_light',
                                 'half_loglight', 'time_half_loglight', 
                                 'hours_below_10','hours_10-100','hours_100-500','hours_over_500','hours_over_1000','amplitude',
                                 'tib_mean_light', 'tib_geometric_mean_light',
                                 'before_tib_mean_light', 'before_tib_geometric_mean_light',
                                 'after_tib_mean_light', 'after_tib_geometric_mean_light',
                                   ])
    
def update_light_table(home_number, all_df, participant, sleep_info, stats_df):
    sleep = sleep_info[sleep_info['participant_id']==participant]
    sleep['day_start'] = pd.to_datetime(sleep['date']).dt.tz_localize(pytz.timezone('Europe/London'))+pd.Timedelta(hours=7)
    sleep['day_end']= sleep['day_start']+ pd.Timedelta(hours=24) - pd.Timedelta(minutes=1)
    sleep['halflight_start'] = pd.to_datetime(sleep['date']).dt.tz_localize(pytz.timezone('Europe/London'))+pd.Timedelta(hours=4)
    sleep['halflight_end']= sleep['halflight_start']+ pd.Timedelta(hours=24) - pd.Timedelta(minutes=1)
    sleep['before_onset'] = sleep['tib_onset'] - pd.Timedelta(hours=3) + pd.Timedelta(minutes=1)
    sleep['after_offset'] = sleep['tib_offset'] + pd.Timedelta(hours=3) - pd.Timedelta(minutes=1)
    
    locations =  ['Bedroom','Lounge','Kitchen','Hallway','Bathroom']
    
    for room in locations: 
        if len(all_df[(all_df['location_name']==room)&(all_df['participant_id']==participant)])!=0:
            new_df = prepare_for_stats(all_df, room, participant)
            print('doing room ',room)
            for i in range(0,len(sleep)):
                daily_data = new_df.loc[new_df['datetime'].between(sleep.iloc[i]['day_start'], sleep.iloc[i]['day_end'])].reset_index(drop=True)
                daily_averages = start_row(participant, room, daily_data, sleep.iloc[i]['date'],sleep.iloc[i]['cluster_number'])    
                halfl_data = new_df.loc[new_df['datetime'].between(sleep.iloc[i]['halflight_start'], sleep.iloc[i]['halflight_end'])].reset_index(drop=True)
                halfl_averages = half_light_values(halfl_data) 
                ntib_data = new_df.loc[new_df['datetime'].between(sleep.iloc[i]['tib_onset'], sleep.iloc[i]['tib_offset']-pd.Timedelta(minutes=1))].reset_index(drop=True)
                ntib_avg = compute_ntib_averages(ntib_data,sleep.iloc[i]['time_in_bed_period'])
                before_ntib_data = new_df.loc[new_df['datetime'].between(sleep.iloc[i]['before_onset'], sleep.iloc[i]['tib_onset'])].reset_index(drop=True)
                before_ntib_avg = compute_before_ntib_averages(before_ntib_data)
                after_ntib_data = new_df.loc[new_df['datetime'].between(sleep.iloc[i]['tib_offset'], sleep.iloc[i]['after_offset'])].reset_index(drop=True)
                after_ntib_avg = compute_after_ntib_averages(after_ntib_data)

                new_row = compute_light_stats(daily_averages, halfl_averages, ntib_avg, before_ntib_avg, after_ntib_avg)
                stats_df = append_row(stats_df, new_row)

        else:
            print('not environmental info at all for ', participant, room)

    return stats_df

In [ ]:
#table = start_light_table()
table = pd.read_parquet('light_metrics.parquet', engine='pyarrow')
for num_participant, participant in enumerate(ids[55:]):
        print('current length table:', len(table))
        print('.....................now doing participant: ', num_participant, participant)
        table = update_light_table(num_participant, light, participant, sleep_df, table) 
        df = pa.Table.from_pandas(table)
        pq.write_table(df, 'light_metrics.parquet', compression='BROTLI')

table

,participant_id,room,date,tib_cluster_number,daily_start,metereological_season,mean_daily_light,geometric_mean_daily_light,half_light,time_half_light,half_loglight,time_half_loglight,hours_below_10,hours_10-100,hours_100-500,hours_over_500,hours_over_1000,amplitude,tib_mean_light,tib_geometric_mean_light,before_tib_mean_light,before_tib_geometric_mean_light,after_tib_mean_light,after_tib_geometric_mean_light
0,Mhy2u,Bedroom,2019-03-31,0,NaT,season,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mhy2u,Bedroom,2019-04-01,1,NaT,season,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mhy2u,Bedroom,2019-04-02,2,NaT,season,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mhy2u,Bedroom,2019-04-03,3,NaT,season,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mhy2u,Bedroom,2019-04-04,5,NaT,season,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239398,QbsYB,Bathroom,2023-06-15,35,2023-06-15 07:00:00+01:00,season,628.009722,86.148835,460355.0,2023-06-15 11:01:00+01:00,1461.029779,2023-06-15 13:06:00+01:00,9.283333,3.383333,2.933333,8.400000,6.183333,2792.0,5.000000,6.000000,34.061111,12.257119,653.600000,26.611305
239399,QbsYB,Bathroom,2023-06-16,36,2023-06-16 07:00:00+01:00,season,476.558333,58.129187,335357.0,2023-06-16 12:21:00+01:00,1198.240449,2023-06-16 14:38:00+01:00,10.533333,2.550000,5.583333,5.333333,4.833333,2749.0,84.572383,21.720135,44.777778,9.091035,819.811111,772.150932
239400,QbsYB,Bathroom,2023-06-17,37,2023-06-17 07:00:00+01:00,season,434.394444,99.805546,315030.0,2023-06-17 12:27:00+01:00,1448.522159,2023-06-17 13:33:00+01:00,7.916667,2.166667,5.550000,8.366667,4.333333,1904.0,55.980861,16.514709,29.000000,7.474386,1413.511111,1143.425710
239401,QbsYB,Bathroom,2023-06-18,38,2023-06-18 07:00:00+01:00,season,453.381944,83.379385,328592.0,2023-06-18 10:59:00+01:00,1387.054062,2023-06-18 13:05:00+01:00,7.966667,4.000000,5.683333,6.350000,4.183333,2337.0,37.676056,14.189641,33.000000,7.679646,1224.127778,975.273507


In [ ]:
br

In [ ]:
len(table[table.isna().any(axis=1)])*100/len(table)

In [ ]:
len(table['participant_id'].unique())

In [ ]:
print(ids)